In [ ]:
pip install pyspark

In [ ]:
# Sử dụng pyspark 
import sys
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions  import *
from pyspark.sql.types import *

# được sử dụng để xác định các đặc điểm của ứng dụng mà chúng ta đang mã hóa, 
#ví dụ như tên của ứng dụng, bộ nhớ được cấp cho các nút Driver và Executor, v.v. 
#Cài đặt Spark.serializer được sử dụng để chọn loại bộ tuần tự hóa dữ liệu 
#(quá trình chuyển đổi dữ liệu thành một cấu trúc khác để lưu trữ và chuyển đến các nút khác nhau trong mạng phân tán hiệu quả 
#đồng thời cho phép xây dựng lại cấu trúc ban đầu của dữ liệu). 
conf = SparkConf().setMaster("local[*]").setAppName("Books")
conf.set("spark.executor.memory", "6G")
conf.set("spark.driver.memory", "2G")
conf.set("spark.executor.cores", "4")
conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
conf.set("spark.default.parallelism", "4")

In [ ]:
# Khởi tạo một sparkSession
spark = SparkSession.builder \
    .config(conf = conf) \
  .appName("spark session example") \
  .getOrCreate()

In [ ]:
#chuyển đổi nó thành RDD, sử dụng đối tượng SparkContext
#SparkContext là điểm đầu vào để sử dụng các chức năng của Spark, vì nó hướng dẫn chương trình truy cập các cụm
books_df = spark.read.option("delimiter", ";").option("header", "true").csv('/content/sample_data/Books.csv')
books_df.show()

+----------+--------------------+--------------------+----+--------------------+
|      ISBN|               Title|              Author|Year|           Publisher|
+----------+--------------------+--------------------+----+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|W. W. Norton & Co...|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|
|0671870432|     PLEADING GUILTY|         Scott Turow|1993|          Audioworks|
|0679425608|Under the Black F...|     David Cordingly|1996|        Random House|
|074322678X|Where You'll Fin

In [ ]:
books_df.select('Title').show()

+--------------------+
|               Title|
+--------------------+
| Classical Mythology|
|        Clara Callan|
|Decision in Normandy|
|Flu: The Story of...|
|The Mummies of Ur...|
|The Kitchen God's...|
|What If?: The Wor...|
|     PLEADING GUILTY|
|Under the Black F...|
|Where You'll Find...|
|Nights Below Stat...|
|Hitler's Secret B...|
|  The Middle Stories|
|            Jane Doe|
|A Second Chicken ...|
|The Witchfinder (...|
|More Cunning Than...|
|Goodbye to the Bu...|
|       The Testament|
|Beloved (Plume Co...|
+--------------------+
only showing top 20 rows



In [ ]:
#tổng số nhà xuất bản trong tập dữ liệu.
books_df.select('Publisher').distinct().count()

16915

In [ ]:
books_df.createOrReplaceTempView('Table') 
df = spark.sql("SELECT Publisher from Table")
df.show()

+--------------------+
|           Publisher|
+--------------------+
|Oxford University...|
|HarperFlamingo Ca...|
|     HarperPerennial|
|Farrar Straus Giroux|
|W. W. Norton & Co...|
|    Putnam Pub Group|
|Berkley Publishin...|
|          Audioworks|
|        Random House|
|            Scribner|
|     Emblem Editions|
|       Citadel Press|
|House of Anansi P...|
|          Mira Books|
|Health Communicat...|
|Brilliance Audio ...|
|Kensington Publis...|
|      River City Pub|
|                Dell|
|               Plume|
+--------------------+
only showing top 20 rows



In [ ]:
#Sử dụng rating để dùng cho việc khuyến nghị
user_ratings_df = spark.read.option("delimiter", ";").option("header", "true").csv('/content/sample_data/Ratings.csv')
# Columns User-ID, ISBN and Book-Rating were in string format, which we convert to int
user_ratings_df.show()



+-------+----------+------+
|User-ID|      ISBN|Rating|
+-------+----------+------+
| 276725|034545104X|     0|
| 276726|0155061224|     5|
| 276727|0446520802|     0|
| 276729|052165615X|     3|
| 276729|0521795028|     6|
| 276733|2080674722|     0|
| 276736|3257224281|     8|
| 276737|0600570967|     6|
| 276744|038550120X|     7|
| 276745| 342310538|    10|
| 276746|0425115801|     0|
| 276746|0449006522|     0|
| 276746|0553561618|     0|
| 276746|055356451X|     0|
| 276746|0786013990|     0|
| 276746|0786014512|     0|
| 276747|0060517794|     9|
| 276747|0451192001|     0|
| 276747|0609801279|     0|
| 276747|0671537458|     9|
+-------+----------+------+
only showing top 20 rows



In [ ]:
ratings_df = user_ratings_df.withColumn("User-ID",
                                        user_ratings_df['User-ID'].\
                                        cast(IntegerType())).\
										withColumn("ISBN", user_ratings_df['ISBN'].\
           								cast(IntegerType())).\
    									withColumn("Rating",\
                                        user_ratings_df['Rating'].\
                                  		cast(IntegerType())).\
                                      na.drop()
ratings_df.show()

+-------+----------+------+
|User-ID|      ISBN|Rating|
+-------+----------+------+
| 276726| 155061224|     5|
| 276727| 446520802|     0|
| 276729| 521795028|     6|
| 276733|2080674722|     0|
| 276737| 600570967|     6|
| 276745| 342310538|    10|
| 276746| 425115801|     0|
| 276746| 449006522|     0|
| 276746| 553561618|     0|
| 276746| 786013990|     0|
| 276746| 786014512|     0|
| 276747|  60517794|     9|
| 276747| 451192001|     0|
| 276747| 609801279|     0|
| 276747| 671537458|     9|
| 276747| 679776818|     8|
| 276747| 943066433|     7|
| 276747|1570231028|     0|
| 276747|1885408226|     7|
| 276748| 747558167|     6|
+-------+----------+------+
only showing top 20 rows



Thuật toán ALS cộng tác sẽ phân rã 2 ma trận là user và iem
-Lấy tích 2 ma trận
-Ưóc lượng rating của user và item thông qua công thức nhân vector



In [ ]:
#Sử dụng phương pháp lọc cộng tác(ALS)
#maxIter: Số lượng vòng lặp tối đa
#regParam: Hệ số kiểm soát regularization
#userCol: tên cột chứa id của user
#itemCol: tên cột chứa id của item
#ratingCol: tên cột chứa giá trị của rating
from datetime import datetime
start_time = datetime.now()
als = ALS(maxIter=5, regParam=0.01, userCol="User-ID", itemCol="ISBN", ratingCol="Rating",coldStartStrategy="drop")#NaN
#fit the model to the ratings
dataframemodel = als.fit(ratings_df)
end_time = datetime.now()

print('Execute time {}'.format(end_time - start_time))

Execute time 0:00:17.862872


In [ ]:
#Số sách tiêu chuẩn quốc tế(ISBN) 

ratings = ratings_df.filter(col('User-ID')==17)
books_df.join(ratings,ratings.ISBN==books_df.ISBN).select(col('User-ID'),col('Title'),col('Author'),col('Year'),col('Rating')).show()

+-------+--------------------+-----------------+----+------+
|User-ID|               Title|           Author|Year|Rating|
+-------+--------------------+-----------------+----+------+
|     17|OUT OF THE SILENT...|       C.S. Lewis|1996|     0|
|     17|Prelude to Founda...|     ISAAC ASIMOV|1989|     0|
|     17|             Prophet| Frank E. Peretti|1992|     3|
|     17|     Winter Solstice|Rosamunde Pilcher|2001|     0|
|     17| Death in the Clouds|  Agatha Christie|1997|     7|
|     17|Piercing the Dark...| Frank E. Peretti|1989|     6|
|     17|Bant/Spec.Last of...|    Louis L'Amour|1987|     5|
+-------+--------------------+-----------------+----+------+



In [ ]:
user_id = [[17]]
conf = SparkConf().setMaster("local[1]").setAppName("book")
sc = SparkContext.getOrCreate(conf=conf)
functiondf = sc.parallelize(user_id).toDF(['User-ID'])
num_rec = 20
recommendations = dataframemodel.recommendForUserSubset(functiondf , num_rec)
recommendations.collect()
recommended_ISBN = [recommendations.collect()[0]['recommendations'][x]['ISBN'] for x in range(0,num_rec)]
recommended_ISBN

[312117787,
 736413057,
 340666269,
 394968808,
 445085428,
 787117900,
 836251822,
 451525817,
 28620283,
 840734247,
 1570820538,
 385326335,
 486241785,
 517884518,
 1580626408,
 786410116,
 882400851,
 906198917,
 843933542,
 590411136]

In [ ]:
rec_df = spark.createDataFrame(recommended_ISBN, IntegerType())
print('Top ',num_rec,' book recommendations for User-ID ',user_id[0][0], ' are:')
books_df.join(rec_df,rec_df.value==books_df.ISBN).select(col('Title'),col('Author'),col('Year')).show()

Top  20  book recommendations for User-ID  17  are:
+--------------------+--------------------+----+
|               Title|              Author|Year|
+--------------------+--------------------+----+
|"The complete mak...|  Barbara Goodfellow|1982|
|               Marin|         Beth Ashley|1993|
|    Timeless Passion|Constance O'Day F...|1994|
|The Glorious Seas...|     Suzanne Clemenz|1989|
|Stokes Hummingbir...|       Donald Stokes|1989|
|When All You've E...|   Harold S. Kushner|1987|
|Campbell's Fabulo...|Campbell Soup Com...|1993|
|Convection Oven C...|        Sunset Books|1984|
|Always A Lady (Si...|         Sharon Sala|2000|
|How to Survive th...|      Melba Colgrove|1991|
|Walt Disney's Win...|         Mouse Works|1994|
|          Annie John|     Jamaica Kincaid|1986|
|   The Runaway Bunny| Margaret Wise Brown|1977|
| The Hidden Alphabet|Laura Vaccaro Seeger|2003|
|Merchant of Venic...| William Shakespeare|1992|
|PCs for Dummies, ...|          Dan Gookin|1997|
|          Headba

https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html